In [2]:
%load_ext autoreload
%autoreload 2

In [28]:


import pandas as pd
import os
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import climetlab as cml
import cfgrib
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from glob import glob


#SD
from mymetrics import compute_rmse, compute_mae, compute_bias, compute_acc
from data_utils import extract_temp_pressure_level, process_and_combine
#SD

In [4]:
data_dir = '/Users/user/Documents_stuff/Ashoka/Capstone_Project/code/ai-models/data/forecast_data/'
fcn_dir = '/Users/user/Documents_stuff/Ashoka/Capstone_Project/code/ai-models/data/forecast_data/fourcastnet'
pangu_dir = '/Users/user/Documents_stuff/Ashoka/Capstone_Project/code/ai-models/data/forecast_data/pangu'
graphcast_dir = '/Users/user/Documents_stuff/Ashoka/Capstone_Project/code/ai-models/data/forecast_data/graphcast'
india_era5_12_steps = '/Users/user/Documents_stuff/Ashoka/Capstone_Project/code/my_graphcast_dir/data/gcp_data/indian_region_era5_date-2022-01-01_res-0.25_levels-13_steps-12.nc'
india_era5_6days = '/Users/user/Documents_stuff/Ashoka/Capstone_Project/code/my_graphcast_dir/data/gcp_data/cds-era5-india-20220101-6days.nc'

In [5]:
lead_time = 120      # hours
init_time = 20220101 # yyyymmdd
start_time = '00'
variables = ['t']    # t: temperature
pressure_levels = [500] # 500 hPa corresponds to approx 5 km above sea level
models = ['FCN', 'pangu'] # neuralgcm

# India specific
lat_min, lat_max = 8.0, 37.0
lon_min, lon_max = 68.0, 97.0

In [64]:
current_pangu_dir = f"{pangu_dir}/init{init_time}-lead{lead_time}-t{start_time}-pangu"
current_fcn_dir = f"{fcn_dir}/init{init_time}-lead{lead_time}-t{start_time}-fcn"

pangu0 = f"{pangu_dir}/init{init_time}-lead{lead_time}-t{start_time}-pangu/out-pangu-0.grib"
fcn0 = f"{fcn_dir}/init{init_time}-lead{lead_time}-t{start_time}-fcn/out-FCNv2-0.grib"
pangu_df = cfgrib.open_datasets(pangu0)
pangu_df = xr.merge(pangu_df, compat='override')
fcn_df = cfgrib.open_datasets(fcn0)
fcn_df = xr.merge(fcn_df, compat='override')
pangu_df

<xarray.Dataset> Size: 287MB
Dimensions:            (latitude: 721, longitude: 1440, isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 8B 2022-01-01
    step               timedelta64[ns] 8B 00:00:00
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time         datetime64[ns] 8B 2022-01-01
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
    meanSea            float64 8B 0.0
Data variables:
    u10                (latitude, longitude) float32 4MB ...
    v10                (latitude, longitude) float32 4MB ...
    t2m                (latitude, longitude) float32 4MB ...
    z                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    t                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    u                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    v                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    q                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    msl                (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [33]:
fcn_df

<xarray.Dataset> Size: 303MB
Dimensions:            (latitude: 721, longitude: 1440, isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 8B 2022-01-01
    step               timedelta64[ns] 8B 00:00:00
    entireAtmosphere   float64 8B 0.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time         datetime64[ns] 8B ...
    heightAboveGround  float64 8B 10.0
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
    meanSea            float64 8B 0.0
    surface            float64 8B 0.0
Data variables: (12/13)
    tcwv               (latitude, longitude) float32 4MB ...
    u10                (latitude, longitude) float32 4MB ...
    v10                (latitude, longitude) float32 4MB ...
    t2m                (latitude, longitude) float32 4MB ...
    u100               (latitude, longitude) float32 4MB ...
    v100               (latitude, longitude) float32 4MB ...
    ...                 ...
    t                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    u                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    v                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    r                  (isobaricInhPa, latitude, longitude) float32 54MB ...
    msl                (latitude, longitude) float32 4MB ...
    sp                 (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [53]:
pangu6 = f"{pangu_dir}/init{init_time}-lead{lead_time}-t{start_time}-pangu/out-pangu-6.grib"
pangu6_df = cfgrib.open_datasets(pangu6)
pangu6_df = xr.merge(pangu6_df, compat='override')
pangu6_df
pangu6_final_df = xr.concat([pangu_df, pangu6_df], dim='step')
pangu6_final_df

<xarray.Dataset> Size: 573MB
Dimensions:            (step: 2, latitude: 721, longitude: 1440,
                        isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 8B 2022-01-01
  * step               (step) timedelta64[ns] 16B 00:00:00 06:00:00
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time         (step) datetime64[ns] 16B 2022-01-01 2022-01-01T06:00:00
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
    meanSea            float64 8B 0.0
Data variables:
    u10                (step, latitude, longitude) float32 8MB 0.06749 ... -0...
    v10                (step, latitude, longitude) float32 8MB 0.008652 ... -...
    t2m                (step, latitude, longitude) float32 8MB 248.4 ... 260.3
    z                  (step, isobaricInhPa, latitude, longitude) float32 108MB ...
    t                  (step, isobaricInhPa, latitude, longitude) float32 108MB ...
    u                  (step, isobaricInhPa, latitude, longitude) float32 108MB ...
    v                  (step, isobaricInhPa, latitude, longitude) float32 108MB ...
    q                  (step, isobaricInhPa, latitude, longitude) float32 108MB ...
    msl                (step, latitude, longitude) float32 8MB 1.029e+05 ... ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [ ]:
model = 'pangu'
final_df = pangu_df
if model == 'pangu':
    for i in range(6, lead_time, 6):
        pangu_df_temp = xr.merge(cfgrib.open_datasets(f"{current_pangu_dir}/out-pangu-{i}.grib"), compat="override")
        if pangu_df_temp is not None:
            print(pangu_df_temp)
            final_df = xr.concat([final_df, pangu_df_temp], dim="step")
del pangu_df_temp
final_df



In [57]:
final_df

<xarray.Dataset> Size: 6GB
Dimensions:            (step: 20, latitude: 721, longitude: 1440,
                        isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 8B 2022-01-01
  * step               (step) timedelta64[ns] 160B 0 days 00:00:00 ... 4 days...
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time         (step) datetime64[ns] 160B 2022-01-01 ... 2022-01-05T1...
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
    meanSea            float64 8B 0.0
Data variables:
    u10                (step, latitude, longitude) float32 83MB 0.06749 ... -...
    v10                (step, latitude, longitude) float32 83MB 0.008652 ... ...
    t2m                (step, latitude, longitude) float32 83MB 248.4 ... 269.7
    z                  (step, isobaricInhPa, latitude, longitude) float32 1GB ...
    t                  (step, isobaricInhPa, latitude, longitude) float32 1GB ...
    u                  (step, isobaricInhPa, latitude, longitude) float32 1GB ...
    v                  (step, isobaricInhPa, latitude, longitude) float32 1GB ...
    q                  (step, isobaricInhPa, latitude, longitude) float32 1GB ...
    msl                (step, latitude, longitude) float32 83MB 1.029e+05 ......
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [ ]:
final_df.to_netcdf(f"{current_pangu_dir}/out-pangu120.nc")


KeyboardInterrupt: 

In [68]:
model = 'fcn'
final_df_fcn_india = fcn_df.sel(
            latitude=slice(lat_max, lat_min),
            longitude=slice(lon_min, lon_max)
        )
if model == 'fcn':
    for i in range(6, lead_time, 6):
        fcn_df_temp = xr.merge(cfgrib.open_datasets(f"{current_fcn_dir}/out-FCNv2-{i}.grib"), compat="override")
        indian_fcn_temp = fcn_df_temp.sel(
            latitude=slice(lat_max, lat_min),
            longitude=slice(lon_min, lon_max)
        )
        if fcn_df_temp is not None:
            print(fcn_df_temp)
            final_df_fcn_india = xr.concat([final_df_fcn_india, indian_fcn_temp], dim="step")
del fcn_df_temp
final_df_fcn_india

<xarray.Dataset> Size: 303MB
Dimensions:            (latitude: 721, longitude: 1440, isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 8B 2022-01-01
    step               timedelta64[ns] 8B 06:00:00
    entireAtmosphere   float64 8B 0.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time         datetime64[ns] 8B ...
    heightAboveGround  float64 8B 10.0
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
    meanSea            float64 8B 0.0
    surface            float64 8B 0.0
Data variables: (12/13)
    tcwv               (latitude, longitude) float32 4MB ...
    u10                (latitude, longitude) float32 4MB ...
    v10                (latitude, longitude) float32 4MB ...
    t2m                (latitude, longitude) float32 4MB ...
    u100               (latitude, longitude) float32 4MB ...
    v100          

<xarray.Dataset> Size: 80MB
Dimensions:            (step: 20, latitude: 117, longitude: 117,
                        isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 8B 2022-01-01
  * step               (step) timedelta64[ns] 160B 0 days 00:00:00 ... 4 days...
    entireAtmosphere   float64 8B 0.0
  * latitude           (latitude) float64 936B 37.0 36.75 36.5 ... 8.5 8.25 8.0
  * longitude          (longitude) float64 936B 68.0 68.25 68.5 ... 96.75 97.0
    valid_time         (step) datetime64[ns] 160B 2022-01-01 ... 2022-01-05T1...
    heightAboveGround  float64 8B 10.0
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
    meanSea            float64 8B 0.0
    surface            float64 8B 0.0
Data variables: (12/13)
    tcwv               (step, latitude, longitude) float32 1MB 14.84 ... 50.61
    u10                (step, latitude, longitude) float32 1MB 1.34 ... -2.486
    v10                (step, latitude, longitude) float32 1MB -0.8947 ... -2...
    t2m                (step, latitude, longitude) float32 1MB 281.8 ... 300.4
    u100               (step, latitude, longitude) float32 1MB 3.448 ... -2.696
    v100               (step, latitude, longitude) float32 1MB -1.042 ... -2.934
    ...                 ...
    t                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    u                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    v                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    r                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    msl                (step, latitude, longitude) float32 1MB 1.019e+05 ... ...
    sp                 (step, latitude, longitude) float32 1MB 9.725e+04 ... ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [62]:
indian_pangu = final_df.sel(
    latitude=slice(lat_max, lat_min),
    longitude=slice(lon_min, lon_max)
)
indian_pangu.to_netcdf(f"{current_pangu_dir}/indian_pangu120.nc")

In [69]:
final_df_fcn_india.to_netcdf(f"{current_fcn_dir}/indian_FCNv2_120.nc")

In [63]:
indian_pangu

<xarray.Dataset> Size: 76MB
Dimensions:            (step: 20, latitude: 117, longitude: 117,
                        isobaricInhPa: 13)
Coordinates:
    time               datetime64[ns] 8B 2022-01-01
  * step               (step) timedelta64[ns] 160B 0 days 00:00:00 ... 4 days...
    heightAboveGround  float64 8B 10.0
  * latitude           (latitude) float64 936B 37.0 36.75 36.5 ... 8.5 8.25 8.0
  * longitude          (longitude) float64 936B 68.0 68.25 68.5 ... 96.75 97.0
    valid_time         (step) datetime64[ns] 160B 2022-01-01 ... 2022-01-05T1...
  * isobaricInhPa      (isobaricInhPa) float64 104B 1e+03 925.0 ... 100.0 50.0
    meanSea            float64 8B 0.0
Data variables:
    u10                (step, latitude, longitude) float32 1MB 1.34 ... -2.56
    v10                (step, latitude, longitude) float32 1MB -0.8947 ... -1...
    t2m                (step, latitude, longitude) float32 1MB 281.8 ... 299.8
    z                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    t                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    u                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    v                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    q                  (step, isobaricInhPa, latitude, longitude) float32 14MB ...
    msl                (step, latitude, longitude) float32 1MB 1.019e+05 ... ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [ ]:
for model in models:
    for level in pressure_levels:
        for var in variables:
            if model == 'FCN':
                model = 'FCNv2'
                forecast_path = os.path.join(fcn_dir, 'init20220101-lead120-t00-fcn')
            elif model == 'pangu':
                forecast_path = os.path.join(pangu_dir, 'init20220101-lead120-t00-pangu')
            elif model == 'graphcast':
                # all the graphcast forecast data is in one file so the processing isn't needed for that
                forecast_path = os.path.join(graphcast_dir, f'{model}_forecast_{var}_{level}hPa_{init_time}_{lead_time}.nc')
            else:
                raise ValueError('Invalid model name')
            # print(forecast_path)
            # data = xr.open_dataset(forecast_path)
            forecast_df = extract_temp_pressure_level(f'{forecast_path}/out-{model}-0.grib', var, level)
            for i in range(6, lead_time+1, 6):
                # print(f"{forecast_path}/out-{model}-{i}.grib")
                # pangu_df_temp = extract_temp_pressure_level(f"{pangu_dir}/out-pangu-{i}.grib",'t', '500')
                # pangu_df = xr.concat([pangu_df, pangu_df_temp], dim="step")
                # del pangu_df_temp
                pass